# CORE Conference Ranking Integration

Jupyter Notebook for the processing and integration of the CORE Conference Ranking.

*The CORE Conference Ranking provides assessments of major conferences in the computing disciplines.The rankings are managed by the CORE Executive Committee, with periodic rounds for submission of requests for addition or reranking of conferences. Decisions are made by academic committees based on objective data requested as part of the submission process.* (source: CORE)

The CORE Ranking is provided in CSV format.
____________________________________________________________

For this process, the following CSV files are needed: ```out_citations_and_conferences_location_ready_v2.csv``` and the CORE Ranking CSVs. 

The first one must be generated running the Notebook ```1 - Citation and Locations Dataset Preparation.ipynb``` that is contained in the same folder as this notebook.<br>
The CORE Ranking CSVs can be downloaded from the [CORE official website](http://portal.core.edu.au/conf-ranks/?search=&by=all&source=CORE2008&sort=atitle&page=1).

In particular, the following operations are going to be executed:
* Opening of the CSV conference citations and locations dataset
* Extraction of the distinct conference series name from the conference citations and locations dataset
* (Sequential) Reading of the CORE CSV files
* Drop of the useless CORE columns
* Filter of the conferences without a rank
* Join between the distinct conference series name and the CORE ratings

Lastly, the processed dataset is going to be saved on disk in CSV format

In [11]:
# Libraries Import
import pandas as pd
import numpy as np
import glob

pd.set_option('display.max_columns', None)

## File Paths
Please set your working directory paths.

In [12]:
# ******************* PATHS ********************+

# Dumps Directory Path
path_file_import = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Import/CORE/'

# CSV Exports Directory Path
path_file_export = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/'

## Read and Preparation of the Citation Dataset

In [13]:
df_citations_and_locations = pd.read_csv(path_file_export + 'out_citations_and_conferences_location_ready_v2.csv', low_memory=False, index_col=[0])
print(f'Successfully Imported the Conference Citations and Locations Ready CSV')

Successfully Imported the Conference Citations and Locations Ready CSV


In [14]:
df_citations_and_locations.head(3)

,CitationCount_COCI,CitationCount_Mag,CitationCount_MagEstimated,ConferenceLocation,ConferenceNormalizedName,ConferenceSeriesNormalizedName,Doi,Year
0,10,12,12,"Austin, Texas, United States",disc 2014,disc,10.1007/978-3-662-45174-8_28,2014
1,5,10,10,"Wrocław, Lower Silesian Voivodeship, Poland",esa 2014,esa,10.1007/978-3-662-44777-2_60,2014
2,11,20,20,"Innsbruck, Tyrol, Austria",enter 2013,enter,10.1007/978-3-319-03973-2_13,2013


### Extracion of the Distinct Conference Series from the Conference and Locations Datasets

In [15]:
df_conference_series = df_citations_and_locations.drop_duplicates(subset="ConferenceSeriesNormalizedName")

#filter of the useless columns
df_conference_series = df_conference_series.drop(df_conference_series.columns.difference(["ConferenceSeriesNormalizedName"]), axis=1)

# drop of the nan row
df_conference_series = df_conference_series.dropna(subset={"ConferenceSeriesNormalizedName"})

# reset of the index
df_conference_series = df_conference_series.reset_index(drop=True)

df_conference_series

,ConferenceSeriesNormalizedName
0,disc
1,esa
2,enter
3,dexa
4,icaisc
...,...
5280,infinity
5281,calculemus
5282,agp
5283,sci


## Read and Processing of the CORE CSVs

First of all, we need to define the CSV header.

**Note**: the most of the columns are useless for our purposes, hence I didn't give them a name.

In [16]:
core_csv_header = ["Conference_Series_Full_Name", "ConferenceSeriesNormalizedName", "Source", "Rank", "Has_Data", "Unnamed_1", "Unnamed_2", "Unnamed_3"]

Read and Processing:

In [17]:
df_conference_series_with_core_rank = df_conference_series.copy()

core_all_csvs = glob.glob(path_file_import + "*.csv")

for current_csv_name_full in core_all_csvs:

    current_csv_name = current_csv_name_full.split("/")[-1]

    # Open the current CSV
    print(f'Currently processing CSV: {current_csv_name}')
    df_core_current_csv = pd.read_csv(current_csv_name_full, names=core_csv_header, low_memory=False)

    # Drop of the useless columns: 
    df_core_current_csv.drop(df_core_current_csv.columns.difference(["ConferenceSeriesNormalizedName", "Rank"]), axis=1, inplace=True)

    # Filter of the unranked conferences
    df_core_current_csv = df_core_current_csv[df_core_current_csv["Rank"].str.contains("Unranked") == False]
    df_core_current_csv = df_core_current_csv[df_core_current_csv["Rank"].str.contains("Australasian") == False]
    df_core_current_csv = df_core_current_csv[df_core_current_csv["Rank"].str.contains("National") == False]
    df_core_current_csv = df_core_current_csv[df_core_current_csv["Rank"].str.contains("journal") == False]

    # Drop of the NaN Conference Acrynyms
    df_core_current_csv = df_core_current_csv.dropna(subset={"ConferenceSeriesNormalizedName"})

    # Making the Conference Acronym to Lowecase
    df_core_current_csv.ConferenceSeriesNormalizedName = df_core_current_csv.ConferenceSeriesNormalizedName.str.lower()

    # Drop of the duplicates (there shouldn't be duplicates)
    df_core_current_csv = df_core_current_csv.drop_duplicates(subset="ConferenceSeriesNormalizedName")

    # Reset of the index
    df_core_current_csv = df_core_current_csv.reset_index(drop=True)

    # Rename of the rank column
    df_core_current_csv = df_core_current_csv.rename(columns={'Rank': (current_csv_name.split('.')[0]) + "_Rank"})

    # Left Join with the Distinct Conferences Dataframe
    df_conference_series_with_core_rank = pd.merge(df_conference_series_with_core_rank, df_core_current_csv, on=['ConferenceSeriesNormalizedName'], how='left')

# Column sort
df_conference_series_with_core_rank = df_conference_series_with_core_rank.reindex(sorted(df_conference_series_with_core_rank.columns), axis=1)

Currently processing CSV: CORE_2013.csv
Currently processing CSV: CORE_2017.csv
Currently processing CSV: ERA_2010.csv
Currently processing CSV: CORE_2014.csv
Currently processing CSV: CORE_2018.csv
Currently processing CSV: CORE_2021.csv
Currently processing CSV: CORE_2020.csv
Currently processing CSV: CORE_2008.csv


Let's check the resulting dataframe:

In [18]:
df_conference_series_with_core_rank

,CORE_2008_Rank,CORE_2013_Rank,CORE_2014_Rank,CORE_2017_Rank,CORE_2018_Rank,CORE_2020_Rank,CORE_2021_Rank,ConferenceSeriesNormalizedName,ERA_2010_Rank
0,A,A,A,A,A,A,A,disc,A
1,A,A,A,A,A,A,A,esa,A
2,NaN,C,C,C,C,C,C,enter,C
3,A,B,B,B,B,B,B,dexa,B
4,NaN,C,C,C,C,NaN,NaN,icaisc,C
...,...,...,...,...,...,...,...,...,...
5280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,infinity,NaN
5281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,calculemus,NaN
5282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,agp,NaN
5283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sci,NaN


## Write of the Final CSVs on Disk

Saving the resulting dataframe on disk in CSV format.

In [19]:
# Write of the resulting CSVs on Disk
df_conference_series_with_core_rank.to_csv(path_file_export + 'out_conference_series_with_core_rank.csv')
print(f'Successfully Exported the Joined CSV to {path_file_export}out_conference_series_with_core_rank.csv')

Successfully Exported the Joined CSV to /Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/out_conference_series_with_core_rank.csv


Check of the Exported CSV to be sure that everything went fine.

In [20]:
# Check of the Exported CSV
df_conference_series_with_core_rank = pd.read_csv(path_file_export + 'out_conference_series_with_core_rank.csv', low_memory=False, index_col=[0])
df_conference_series_with_core_rank

,CORE_2008_Rank,CORE_2013_Rank,CORE_2014_Rank,CORE_2017_Rank,CORE_2018_Rank,CORE_2020_Rank,CORE_2021_Rank,ConferenceSeriesNormalizedName,ERA_2010_Rank
0,A,A,A,A,A,A,A,disc,A
1,A,A,A,A,A,A,A,esa,A
2,NaN,C,C,C,C,C,C,enter,C
3,A,B,B,B,B,B,B,dexa,B
4,NaN,C,C,C,C,NaN,NaN,icaisc,C
...,...,...,...,...,...,...,...,...,...
5280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,infinity,NaN
5281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,calculemus,NaN
5282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,agp,NaN
5283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sci,NaN
